In [12]:
import numpy as np

#starting_board = []

Första approach: två 8x8 plan på varandra
 + Läsbar, men
     - Man måste ändå hålla reda på att ena planet är förskjutet eftersom det representerar kryssen mellan rutorna
     - Detta plan har också en onödig rad och kolumn
 - Väldigt o-optimerad

In [13]:
test = np.zeros((2,8,8))
print(test)

[[[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]

 [[0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 0. 0. 0.]]]


In [14]:
test[0]

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0.]])

För att komma fram till en mera optimal representation tror jag att jag först borde börja med att representare bägge "sidor" av planen skilt.

Från checkers implementationen som jag har lånat vet jag redan att den planen kan komprimeras till 4x8 eftersom pjäserna endast rör sig diagonalt



In [15]:
np.zeros((7,7))

array([[0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.]])

In [16]:
# Lånat från checkers:
def expand(board):
	b = np.zeros((8, 8), dtype='b')
	for i in range(0, 8):
		if (i%2 == 0):
			b[i] = np.array([0, board[i*4], 0, board[i*4 + 1], 0, board[i*4 + 2], 0, board[i*4 + 3]])
		else:
			b[i] = np.array([board[i*4], 0, board[i*4 + 1], 0, board[i*4 + 2], 0, board[i*4 + 3], 0])
	return b

def compress(board):
	b = np.zeros((1,32), dtype='b')
	for i in range(0, 8):
		if (i%2 == 0):
			b[0, i*4 : i*4+4] = np.array([board[i, 1], board[i, 3], board[i, 5], board[i, 7]])
		else:
			b[0, i*4 : i*4+4] = np.array([board[i, 0], board[i, 2], board[i, 4], board[i, 6]])
	return b

In [17]:
import numpy as np
b = np.zeros((1, 8), dtype='b')
b[0,0] = 4

In [18]:
b[0,0]

4

In [19]:
manual_diagonals = [[4,5],[5,6],[6,7],[7],[0,8],[0,1,8,9],[1,2,9,10],[2,3,10,11],[12,13,4,5],[13,14,5,6]]

In [20]:
def diagonal_steps(position):
    steps = []
    # Beacuse the checker board is compressed, diagonals work differently on add and even rows
    if (position // 4) % 2 == 0:
        if not ((position + 5) % 8 == 0):
            steps.append(position - 5)
            steps.append(position + 5)
        steps.append(position - 4)
        steps.append(position + 4)
    else:
        if not ((position + 4) % 8 == 0):
            steps.append(position - 5)
            steps.append(position + 3)
        steps.append(position - 4)
        steps.append(position + 4)
    i = 0
    while i < len(steps):
        if steps[i] < 0 or steps[i] > 31:
            steps.pop(i)
        else: i = i + 1
    return steps    

In [22]:
auto_steps = []
for n in range(32):
    auto_steps.append(diagonal_steps(n))

In [23]:
auto_steps

[[5, 4],
 [6, 5],
 [7, 6],
 [7],
 [0, 8],
 [0, 8, 1, 9],
 [1, 9, 2, 10],
 [2, 10, 3, 11],
 [3, 13, 4, 12],
 [4, 14, 5, 13],
 [5, 15, 6, 14],
 [7, 15],
 [8, 16],
 [8, 16, 9, 17],
 [9, 17, 10, 18],
 [10, 18, 11, 19],
 [11, 21, 12, 20],
 [12, 22, 13, 21],
 [13, 23, 14, 22],
 [15, 23],
 [16, 24],
 [16, 24, 17, 25],
 [17, 25, 18, 26],
 [18, 26, 19, 27],
 [19, 29, 20, 28],
 [20, 30, 21, 29],
 [21, 31, 22, 30],
 [23, 31],
 [24],
 [24, 25],
 [25, 26],
 [26, 27]]

In [24]:
auto_steps[28]

[24]

In [25]:
manual_neighbors = [[1,7],[0,2,8]]

In [32]:
def neighbors(point, offset):
    # offset is added to neigbors because the go board will be appended to the checkers board
    neighbors = []
    if not (point % 7 == 0):
        neighbors.append(point - 1 + offset)
    if not ((point + 1) % 7 == 0):
        neighbors.append(point + 1 + offset)
    neighbors.append(point - 7 + offset)
    neighbors.append(point + 7 + offset)
    i = 0
    while i < len(neighbors):
        if neighbors[i] - offset < 0 or neighbors[i] - offset > 48:
            neighbors.pop(i)
        else: i = i + 1
    return neighbors   
    

In [35]:
auto_neighbors = []
for n in range(49):
    auto_neighbors.append(neighbors(n, 32))

In [36]:
auto_neighbors

[[33, 39],
 [32, 34, 40],
 [33, 35, 41],
 [34, 36, 42],
 [35, 37, 43],
 [36, 38, 44],
 [37, 45],
 [40, 32, 46],
 [39, 41, 33, 47],
 [40, 42, 34, 48],
 [41, 43, 35, 49],
 [42, 44, 36, 50],
 [43, 45, 37, 51],
 [44, 38, 52],
 [47, 39, 53],
 [46, 48, 40, 54],
 [47, 49, 41, 55],
 [48, 50, 42, 56],
 [49, 51, 43, 57],
 [50, 52, 44, 58],
 [51, 45, 59],
 [54, 46, 60],
 [53, 55, 47, 61],
 [54, 56, 48, 62],
 [55, 57, 49, 63],
 [56, 58, 50, 64],
 [57, 59, 51, 65],
 [58, 52, 66],
 [61, 53, 67],
 [60, 62, 54, 68],
 [61, 63, 55, 69],
 [62, 64, 56, 70],
 [63, 65, 57, 71],
 [64, 66, 58, 72],
 [65, 59, 73],
 [68, 60, 74],
 [67, 69, 61, 75],
 [68, 70, 62, 76],
 [69, 71, 63, 77],
 [70, 72, 64, 78],
 [71, 73, 65, 79],
 [72, 66, 80],
 [75, 67],
 [74, 76, 68],
 [75, 77, 69],
 [76, 78, 70],
 [77, 79, 71],
 [78, 80, 72],
 [79, 73]]